In [ ]:
import sys; sys.path.insert(0, '..')

In [ ]:
import geopandas
import pandas as pd
import folium
import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta

import osmnx as ox

In [ ]:
df = geopandas.read_parquet("data/checkpoints/joined.parquet")

In [ ]:
with open("data/filter.json") as f:
    filter_out = json.load(f)

In [ ]:
d = df[['Activity ID', 'Activity Date', 'Activity Type', 'Activity Name', 'Distance', 'geometry', 'city']].head(3)

In [ ]:
d

In [ ]:
d['geometry'] = d['geometry'].apply(lambda x: str(x)[0:37] + "...")

In [ ]:
print(d.to_markdown())

In [ ]:
df = pd.read_csv("data/strava/activities.csv")

In [ ]:
d = df[['Activity ID', 'Activity Date', 'Activity Type', 'Activity Name', 'Distance', 'Filename']].head(3)

In [ ]:
print(d.to_markdown())

## Boston

In [ ]:
df = geopandas.read_parquet("data/checkpoints/joined.parquet")

In [ ]:
df = df[df["city"] == "Boston"].reset_index(drop=True)

In [ ]:
# drop hand-picked activities
df = df[~df["Activity ID"].isin(filter_out["Boston"])]

In [ ]:
def reverse(coords):
    return [(x[1], x[0]) for x in coords]

# make map
m = folium.Map(location=(42, -71), zoom_start=8, tiles="cartodb positron")

for i, row in df.iterrows():
    pts = reverse(list(row["geometry"].coords))
    m.add_child(folium.PolyLine(
        locations=pts, weight=3, tooltip=row["Activity ID"],
    ))

In [ ]:
# m

In [ ]:
df['Distance'].sum()

In [ ]:
d_start = df['Activity Date'].head(1).iloc[0]
d_start

In [ ]:
d_end = df['Activity Date'].tail(1).iloc[0]
d_end

In [ ]:
d_start = datetime.strptime(d_start, "%Y-%m-%d %H:%M:%S")
d_end = datetime.strptime(d_end, "%Y-%m-%d %H:%M:%S")

d_end - d_start

In [ ]:
d = df.loc[0, 'Activity Date']
date_object = datetime.strptime(d, "%Y-%m-%d %H:%M:%S")

dates = [date_object]
new_date = date_object
for i in range(len(df)-1):
    new_date = new_date + timedelta(days=1)
    dates.append(new_date)

dates = [str(x) for x in dates]
df['Activity Date'] = dates

In [ ]:
df = df.rename(
    columns={
        "Activity ID": "id",
        "Activity Date": "datetime",
        "Activity Type": "type",
        "Activity Name": "name",
        "Elapsed Time": "duration",
        "Distance": "distance",
        "new_path": "filepath"
    }
)

In [ ]:
df.to_file("out/Boston")

## Denver

In [ ]:
df = geopandas.read_parquet("data/checkpoints/joined.parquet")

In [ ]:
df = df[df["city"] == "Denver"].reset_index(drop=True)

In [ ]:
# drop hand-picked activities
# df = df[~df["Activity ID"].isin(filter_out["Denver"])]

In [ ]:
def reverse(coords):
    return [(x[1], x[0]) for x in coords]

# make map
m = folium.Map(location=(39.75, -104.9), zoom_start=8, tiles="cartodb positron")

for i, row in df.iterrows():
    pts = reverse(list(row["geometry"].coords))
    m.add_child(folium.PolyLine(
        locations=pts, weight=3, tooltip=row["Activity ID"],
    ))

In [ ]:
# m

In [ ]:
df['Activity Date'] = pd.to_datetime(df['Activity Date'])

In [ ]:
len(df)

In [ ]:
df["Distance"].sum()

In [ ]:
df = df[df['Activity Date'] < '2022-11-15']

In [ ]:
d_start = datetime.strptime('2021-08-06', "%Y-%m-%d")
d_end = datetime.strptime('2022-11-14', "%Y-%m-%d")

d_end - d_start

In [ ]:
df = pd.DataFrame(df[['Activity Date', 'Distance']])

In [ ]:
# df.plot(kind="hist", x='Activity Date', y='Distance')

### Timestamps

In [ ]:
# df

In [ ]:
d = df.loc[0, 'Activity Date']
date_object = datetime.strptime(d, "%Y-%m-%d %H:%M:%S")
date_object

In [ ]:
dates = [date_object]
new_date = date_object
for i in range(len(df)-1):
    new_date = new_date + timedelta(days=1)
    dates.append(new_date)

In [ ]:
dates = [str(x) for x in dates]
df['Activity Date'] = dates

### Export

In [ ]:
df = df.rename(
    columns={
        "Activity ID": "id",
        "Activity Date": "datetime",
        "Activity Type": "type",
        "Activity Name": "name",
        "Elapsed Time": "duration",
        "Distance": "distance",
        "new_path": "filepath"
    }
)

In [ ]:
df.to_file("out/Denver")

## Gothenburg

In [ ]:
df = geopandas.read_parquet("data/checkpoints/joined.parquet")

In [ ]:
df = df[df["city"] == "Gothenburg"].reset_index(drop=True)

In [ ]:
# drop hand-picked activities
df = df[~df["Activity ID"].isin(filter_out["Gothenburg"])]

In [ ]:
# filter out recent activities
# df = df[df['Activity Date'] < '2021-05-24']

In [ ]:
d_start = datetime.strptime('2020-04-05', "%Y-%m-%d")
d_end = datetime.strptime('2021-04-23', "%Y-%m-%d")

d_end - d_start

In [ ]:
df["Activity Date"] = pd.to_datetime(df["Activity Date"])

In [ ]:
df = df[df["Activity Date"] < "2021-04-24"]

In [ ]:
len(df)

In [ ]:
df["Distance"].sum()

In [ ]:
df

In [ ]:
city_name = "Göteborg, Sweden"
city_gdf = ox.geocode_to_gdf(city_name)

In [ ]:
city_gdf.bbox_north

In [ ]:
city_gdf.bbox_south

In [ ]:
len(df)

In [ ]:
df = df.to_crs("EPSG:3006")

In [ ]:
df.head(1).plot()

In [ ]:
df[['x', 'y']] = pd.DataFrame(df['geometry'].apply(lambda x: x.xy).tolist(), index=df.index)

In [ ]:
np.array(df["x"].apply(lambda x: np.array(x).max())).max()

In [ ]:
np.array(df["y"].apply(lambda x: np.array(x).max())).max()

In [ ]:
np.array(df["x"].apply(lambda x: np.array(x).min())).min()

In [ ]:
np.array(df["y"].apply(lambda x: np.array(x).min())).min()

In [ ]:
384557 - 295637

In [ ]:
l = df.iloc[0]["geometry"]

In [ ]:
x, y = l.xy

In [ ]:
def plot(df):
    # Create the figure and axis
    # fig, ax = plt.subplots(figsize=(8, 6))
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Set the background color to black
    # fig.patch.set_facecolor('black')  # Figure background
    ax.set_facecolor('black')  # Axis background
    
    # Set specific bounds for x and y axes
    x_lim, y_lim = 295637, 6357281
    ax.set_xlim(x_lim, x_lim+40000)
    ax.set_ylim(y_lim, y_lim+40000) 
    
    # Set aspect ratio to equal
    ax.set_aspect('equal', adjustable='box')
    
    # Label the axes
    ax.set_xlabel("X-Axis")
    ax.set_ylabel("Y-Axis")
    
    # add a line
    for i, row in df.iterrows():
        geo = row["geometry"]
        x, y = geo.xy
        ax.plot(x, y, linewidth=0.26, color="#C7D3DD")
    
    # Customize ticks to white
    # ax.tick_params(colors='white')
    
    # Display the plot
    plt.grid(False)
    plt.show()

    return fig, ax

In [ ]:
fig, ax = plot(df)

In [ ]:
# fig.savefig("img.png", dpi=300)

In [ ]:
# def plot_sample(df, n=1):
#     sample = geopandas.GeoDataFrame(df.iloc[0:n])
#     sample["new"] = False
#     sample.loc[len(sample)-1, "new"] = True
#     print(sample["new"].head())
#     sample.plot(
#         column="new",
#         cmap="Set3",
#     )

In [ ]:
# plot_sample(df, 12)

In [ ]:
# def reverse(coords):
#     return [(x[1], x[0]) for x in coords]

# # make map
# m = folium.Map(location=(57.7, 11.56), zoom_start=9, tiles="cartodb positron")

# for i, row in df.iterrows():
#     pts = reverse(list(row["geometry"].coords))
#     m.add_child(folium.PolyLine(
#         locations=pts, weight=3, tooltip=row["Activity ID"],
#     ))

In [ ]:
# m

In [ ]:
d = df.loc[0, 'Activity Date']
date_object = datetime.strptime(d, "%Y-%m-%d %H:%M:%S")
date_object

In [ ]:
dates = [date_object]
new_date = date_object
for i in range(len(df)-1):
    new_date = new_date + timedelta(days=1)
    dates.append(new_date)

In [ ]:
dates = [str(x) for x in dates]
df['Activity Date'] = dates

In [ ]:
df.head()

In [ ]:
df = df.rename(
    columns={
        "Activity ID": "id",
        "Activity Date": "datetime",
        "Activity Type": "type",
        "Activity Name": "name",
        "Elapsed Time": "duration",
        "Distance": "distance",
        "new_path": "filepath"
    }
)

In [ ]:
df.to_file("out/gothenburg")

In [ ]:
df